# Profiling main skforecast classes and methods

In [140]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [141]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset

%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [142]:
series_1 = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_1')
series_2 = pd.Series([20, 30, 40, 50, 60, 70, 80, 90, 100, 110], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_2')
series = pd.concat([series_1, series_2], axis=1).melt(var_name='series', value_name='value', ignore_index=False)
series = series.rename_axis('datetime')
print(series.index.freq)
series

None


,series,value
datetime,,
2000-01-01,series_1,1
2000-01-02,series_1,2
2000-01-03,series_1,3
2000-01-04,series_1,4
2000-01-05,series_1,5
2000-01-06,series_1,6
2000-01-07,series_1,7
2000-01-08,series_1,8
2000-01-09,series_1,9


In [ ]:
series = series.groupby('series').apply(lambda x: x.asfreq('D'), include_groups=False)
series

value
series   datetime         
series_1 2000-01-01      1
         2000-01-02      2
         2000-01-03      3
         2000-01-04      4
         2000-01-05      5
         2000-01-06      6
         2000-01-07      7
         2000-01-08      8
         2000-01-09      9
         2000-01-10     10
series_2 2000-01-01     20
         2000-01-02     30
         2000-01-03     40
         2000-01-04     50
         2000-01-05     60
         2000-01-06     70
         2000-01-07     80
         2000-01-08     90
         2000-01-09    100
         2000-01-10    110

In [144]:
series.loc['series_1'].index

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10'],
              dtype='datetime64[ns]', name='datetime', freq='D')

In [154]:
series_1 = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_1')
series_2 = pd.Series([20, 30, 40, 50, 60, 70, 80, 90, 100, 110], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_2')
series = pd.concat([series_1, series_2], axis=1).melt(var_name='series', value_name='value', ignore_index=False)
series = series.rename_axis('datetime')
row_to_drop = '2000-01-02'
series_dropped = series.loc[series.index != row_to_drop]
series_dropped

,series,value
datetime,,
2000-01-01,series_1,1
2000-01-03,series_1,3
2000-01-04,series_1,4
2000-01-05,series_1,5
2000-01-06,series_1,6
2000-01-07,series_1,7
2000-01-08,series_1,8
2000-01-09,series_1,9
2000-01-10,series_1,10


In [155]:
series_dropped = series_dropped.groupby('series').apply(lambda x: x.asfreq('D'), include_groups=False)
series_dropped

value
series   datetime         
series_1 2000-01-01    1.0
         2000-01-02    NaN
         2000-01-03    3.0
         2000-01-04    4.0
         2000-01-05    5.0
         2000-01-06    6.0
         2000-01-07    7.0
         2000-01-08    8.0
         2000-01-09    9.0
         2000-01-10   10.0
series_2 2000-01-01   20.0
         2000-01-02    NaN
         2000-01-03   40.0
         2000-01-04   50.0
         2000-01-05   60.0
         2000-01-06   70.0
         2000-01-07   80.0
         2000-01-08   90.0
         2000-01-09  100.0
         2000-01-10  110.0

In [162]:
series_1 = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_1')
series_2 = pd.Series([20, 30, 40, 50, 60, 70, 80, 90, 100, 110], index=pd.date_range(start='2000-01-01', periods=10, freq='D'), name='series_2')
series = pd.concat([series_1, series_2], axis=1).melt(var_name='series', value_name='value', ignore_index=False)
series = series.rename_axis('datetime')
series = series.groupby('series').apply(lambda x: x.asfreq('D'), include_groups=False)
series

exog = pd.DataFrame({
    'exog_1': [1, 2, 3, 4, 5, 6, 7, 8],
    'exog_2': [10, 20, 30, 40, 50, 60, 70, 80]
}, index=pd.date_range(start='2000-01-01', periods=8, freq='D'))
exog = exog.rename_axis('datetime')

# merge series and exog
series_exog = pd.merge(series, exog, left_index=True, right_index=True, how='left')
series_exog

value  exog_1  exog_2
series   datetime                         
series_1 2000-01-01      1     1.0    10.0
         2000-01-02      2     2.0    20.0
         2000-01-03      3     3.0    30.0
         2000-01-04      4     4.0    40.0
         2000-01-05      5     5.0    50.0
         2000-01-06      6     6.0    60.0
         2000-01-07      7     7.0    70.0
         2000-01-08      8     8.0    80.0
         2000-01-09      9     NaN     NaN
         2000-01-10     10     NaN     NaN
series_2 2000-01-01     20     1.0    10.0
         2000-01-02     30     2.0    20.0
         2000-01-03     40     3.0    30.0
         2000-01-04     50     4.0    40.0
         2000-01-05     60     5.0    50.0
         2000-01-06     70     6.0    60.0
         2000-01-07     80     7.0    70.0
         2000-01-08     90     8.0    80.0
         2000-01-09    100     NaN     NaN
         2000-01-10    110     NaN     NaN